# Linear Regression Regularization

## I. Linear Regression Basics

Linear regression is a tool often underrated tool in the machine learning toolkit.

The loss function most often used in linear regression is the sum of squared residuals (SSR). Minimizing this value is equivalent to minimizing the mean squared error (MSE).

The loss function in matrix form is:

# $\text{min } (y - X\hat{\beta})^T(y - X\hat{\beta})$

Using summation notation, the loss function is:

# $\text{min } \Sigma_{i=1}^n (y_i - \Sigma^k_{j=1} (\hat{\beta}_j x_{ij}))^2$

where $y$ is the column vector of true values, $X$ is the design matrix, and $\hat{\beta}$ is the column vector of coefficient estimates in the linear model.